In [2]:
# config.py

import os
import numpy as np
import tensorflow as tf
from google.colab import drive


# Mounting drive
# This will require authentication
drive.mount('/content/drive')


SEED = 42
EPOCHS = 100
BATCH_SIZE = 32
IMG_HEIGHT = 192
IMG_WIDTH = 256

input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)

APP_PATH = '/content/drive/MyDrive/unik/cataract-mat-model'

def get_full_path(*pathes):
    path = ''.join(map(str, pathes))
    return os.path.join(APP_PATH, path)

# # cataract dataset
# IMG_ROOT = 'input/dataset/'
# IMG_DIR = [IMG_ROOT+'1_normal', 
#            IMG_ROOT+'2_cataract', 
#            IMG_ROOT+'2_glaucoma', 
#            IMG_ROOT+'3_retina_disease']

# ocular-disease-recognition dataset
OCU_ROOT = 'input/ODIR_5K/'
OCU_IMG_ROOT = 'input/ODIR_5K/Training_Images/'

# FULL_IMG_ROOT = get_full_path(IMG_ROOT)
FULL_OCU_ROOT = get_full_path(OCU_ROOT)
FULL_OCU_IMG_ROOT = get_full_path(OCU_IMG_ROOT)
# FULL_OCU_DATA_ROOT = get_full_path(FULL_OCU_ROOT, "data.xlsx")

# ocu_df = pd.read_excel(
#      FULL_OCU_DATA_ROOT,
#      engine='openpyxl',
# )


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# predict_on_efficient_model.py

"""
  Need to be install
"""
!pip install -q efficientnet >> /dev/null

# !pip install openpyxl

import os, glob, cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import get_custom_objects
import efficientnet.tfkeras as efn

# from config import *


def create_one_example(path, img_width, img_height):
    img = plt.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_width, img_height))
        
    return img

def normalize_example(img):
    return img[np.newaxis, :]/255.0


"""
	Loading efficientnet-b0 model and predicting on separate example
"""

effi_model_path = get_full_path('efficientnet-b0_model.h5')

loaded_efficientnet_model = tf.keras.models.load_model(effi_model_path)

effi_example_path = get_full_path(OCU_IMG_ROOT, '0_left.jpg')
effi_example = create_one_example(effi_example_path, IMG_WIDTH, IMG_HEIGHT)
effi_example = normalize_example(effi_example)

effi_result = loaded_efficientnet_model.predict(effi_example)
try:
  effi_result_percent = effi_result[0][1] * 100
except IndexError as e:
  print('Invalid data: {}'.format(e))

print('Efficientnet-b0 Model\nDiagnosis: cataract with {:.2f}% probability'.format(effi_result_percent))

Efficientnet-b0 Model
Diagnosis: cataract with 99.62% probability
